In [1]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')
sys.path.insert(0,'..')

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:

from audio.redis import *
from audio.audio import *
#from audio.process import *

from pathlib import Path
import json

import uuid

import pandas as pd

import asyncio
import redis.asyncio as aioredis
from dataclasses import dataclass
import numpy as np

import json
import time
pd.set_option('display.max_colwidth', None)

In [4]:

async def get_next_chunk_start(redis_inner_client, audio_name, length,shift):
    diarization = Diarisation(audio_name, redis_inner_client)
    await diarization.get()
    diarization_result = diarization.data

    if len(diarization_result)>0:
        last_speech = diarization_result[-1]

        ended_silence = length - last_speech['end']
        print(ended_silence)
        if ended_silence<2:
            print('interrupted')
            return last_speech['start']+shift
        

        else:
            print('non-interrupted') 
            return last_speech['end']+shift



    else: return None



async def diarize(client_id, audio_name, shift, redis_inner_client):
    await redis_inner_client.lpush('Audio2DiarizeQueue', f'{audio_name}:{client_id}')
    done = await redis_inner_client.brpop(f'DiarizeReady:{audio_name}')
    diarization = Diarisation(audio_name, redis_inner_client)
    await diarization.get()
    return diarization.data

    


async def transcribe(audio_name, redis_inner_client):
    await redis_inner_client.lpush('Audio2TranscribeQueue', f'{audio_name}:{client_id}')
    _,done = await redis_inner_client.brpop(f'TranscribeReady:{audio_name}')
    transcription =  Transcript(audio_name,redis_inner_client)
    await transcription.get()
    return transcription.data


In [5]:
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'
import subprocess
from pydub import AudioSegment
import io
from time import sleep

from audio.redis import Transcript

In [6]:
redis_inner_client = await get_inner_redis()
redis_stream_client = await get_stream_redis()
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'

In [7]:
await get_connections_df('initialFeed_audio',redis_stream_client)

,queue,length


In [8]:
connections =  await get_connections('initialFeed_audio',redis_stream_client)
connection_ids = [c.replace('initialFeed_audio:','') for c in connections]


connections =  await get_connections('initialFeed_audio',redis_stream_client)

In [9]:
#[await redis_stream_client.delete(c) for c in connections]

In [10]:
connection_id = 'david_audio'#'7c781102-9ee6-4f3e-af7b-54ca79adb82f'

In [11]:

path = f'/app/testdata/{connection_id}.webm'

In [12]:
def expand_over_timeindex(df,shift,column):
    if len(df)>0:
        expanded_df = pd.DataFrame(columns=[column, 'time'])
        for index, row in df.iterrows():
            time_range = np.arange(row['start'] * 100, row['end'] * 100, 1).astype(int)

            temp_df = pd.DataFrame({
                column: row[column],
                'time': time_range
            })
            expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)
        expanded_df['conv_time'] = (expanded_df['time'] + shift*100).astype(int)

        return expanded_df.sort_values("time")

In [13]:
on_start = False

In [223]:

#on_start = True

In [224]:

step = 5
max_length=20

In [225]:
start = 0

In [226]:
redis_stream_client = await get_stream_redis()
redis_inner_client = await get_inner_redis()

In [227]:
diarizations = []
transcriptions = []

In [228]:
def clean_diarization_result(diarization_result,min_len=0.75):
    df = pd.DataFrame(diarization_result)
    df['speaker'] = np.where(df['end']-df['start']<min_len,np.nan,df['speaker'])
    return df.fillna(method='ffill').fillna(method='bfill').to_dict(orient='records')

In [243]:
diarization_result

[{'start': 0.008488964346349746,
  'end': 0.5178268251273346,
  'speaker': 'cf8ef7d6-bca6-4ca6-8c7b-630b4ee98acc'},
 {'start': 1.0271646859083192,
  'end': 2.0288624787775893,
  'speaker': 'cf8ef7d6-bca6-4ca6-8c7b-630b4ee98acc'},
 {'start': 2.67402376910017,
  'end': 4.168081494057725,
  'speaker': 'cf8ef7d6-bca6-4ca6-8c7b-630b4ee98acc'},
 {'start': 5.679117147707979,
  'end': 6.3921901528013585,
  'speaker': 'cf8ef7d6-bca6-4ca6-8c7b-630b4ee98acc'},
 {'start': 7.071307300509337,
  'end': 9.312393887945671,
  'speaker': 'cf8ef7d6-bca6-4ca6-8c7b-630b4ee98acc'},
 {'start': 12.130730050933787,
  'end': 13.04753820033956,
  'speaker': 'cf8ef7d6-bca6-4ca6-8c7b-630b4ee98acc'},
 {'start': 14.219015280135824,
  'end': 18.78607809847199,
  'speaker': 'cf8ef7d6-bca6-4ca6-8c7b-630b4ee98acc'},
 {'start': 18.78607809847199,
  'end': 19.193548387096776,
  'speaker': 'cf8ef7d6-bca6-4ca6-8c7b-630b4ee98acc'},
 {'start': 19.244482173174873,
  'end': 19.65195246179966,
  'speaker': 'cf8ef7d6-bca6-4ca6-8c7

In [255]:
await writestream2file(connection_id,redis_stream_client)
audio_slicer = await AudioSlicer.from_ffmpeg_slice(path,start,max_length)

slice_duration = audio_slicer.audio.duration_seconds
print(slice_duration)

if slice_duration > step:

    audio_data = await audio_slicer.export_data()
    audio_name = str(uuid.uuid4())
    audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
    await audio.save()

    diarization_result, transcription_result = await asyncio.gather(
            diarize(client_id, audio_name, start,redis_inner_client),
            transcribe(audio_name,redis_inner_client)
        )
    
if len(diarization_result)>0:
    diarization_result = clean_diarization_result(diarization_result)


    diarization_df = expand_over_timeindex(pd.DataFrame(diarization_result),start,'speaker')
    diarization_df['shift'] = start
    transcription_df = expand_over_timeindex(pd.concat([pd.DataFrame(t) for t in transcription_result]),start,'word')
    transcription_df['shift'] = start
    diarizations.append(diarization_df)
    transcriptions.append(transcription_df)
    start_ = await get_next_chunk_start(redis_inner_client, audio_name, slice_duration,start)
    start = start_ if start_ else start+slice_duration
    diarization_df = pd.concat(diarizations).sort_values('shift',ascending=False).drop_duplicates('conv_time').sort_values('conv_time')
    transcription_df = pd.concat(transcriptions).sort_values('shift',ascending=False).drop_duplicates('conv_time').sort_values('conv_time')
    df = diarization_df[['conv_time',"speaker"]].set_index('conv_time').join(transcription_df[['conv_time','word']].set_index('conv_time')).reset_index().dropna()
    df['cleaned_text'] = df['word'].str.replace('[^a-zA-Z]', '', regex=True)
    df['word_switch'] = df['cleaned_text']!=df['cleaned_text'].shift()
    df = df[df['word_switch']].drop(columns = ['word_switch','cleaned_text'])
    df['speaker_switch'] = df['speaker']!=df['speaker'].shift()
    df['speaker_switch'] = df['speaker_switch'].cumsum()
    df = df.reset_index(drop=True)
    speakers = df.drop_duplicates('speaker_switch').set_index('speaker_switch')['speaker'].to_frame('speaker')
    r = (df.groupby('speaker_switch')['word'].apply(lambda x:''.join(x)).to_frame('text')
    .join(speakers)
    .join(df.groupby('speaker_switch')['conv_time'].min().to_frame('min'))
    .join(df.groupby('speaker_switch')['conv_time'].max().to_frame('max')))
    r['span'] = r['max']-r['min']

    max_length = 20
else:
    max_length+=step
r.tail(5)

20.04
None
0.6596943972835305
interrupted


/tmp/ipykernel_1550100/506151659.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.fillna(method='ffill').fillna(method='bfill').to_dict(orient='records')


,text,speaker,min,max,span
speaker_switch,,,,,
1,"But this life cycle is indicative only, right? So in other words, you could take any life cycle and do what I'm going to do here. So initiation, we typically have discovery, design.",cf8ef7d6-bca6-4ca6-8c7b-630b4ee98acc,50,1307,1257
2,test implement,5062be38-0ba9-4e87-94e3-e3365454f932,2294,2304,10
3,"call it benefits relaxation but it's a bit of a in fact we'll have to put that in there because it kind of happens and close now that's a fairly sort of waterfall timer if you have um agile projects what we tend to do is we'll iterate around this cycle here Felly, os yw'n rhan agorol o waith, byddwn ni bob amser yn cyflwyno'r tîm, byddwn ni'r tîm, byddwn'n gweithio arnyn nhw, byddwn ni'n ysgrifennu drwy bob sbrin, neu o fewn, efallai, dros ddau sbrin, byddwn'n gwneud rhywfaint o ddysgu, adeiladu, testio a chyflwyno o amgylcheddau, ac and eventually you reach cyflawni reach an end and then you say right the work's done we'll close so this is both waterfall and it will do agile okay what tends to happen uh if you look how these things flow you've got uh We'll have a series of calls, uh either project calls, we may have stand-ups, we may have governance meetings, governance, so So those things cover everything from resourcing, finance, project reviews, and all of those sort of fall under the category of meetings. Now, meetings, as you probably know, typically will have an agenda. They'll have some inputs and outputs. So by inputs and outputs, I really mean defined inputs. It might be a document, Uh, some data and it'll have some outputs. Now the outputs of what's interesting because you'll get minutes. Sorry, give me one second. I got somebody at front door.",cf8ef7d6-bca6-4ca6-8c7b-630b4ee98acc,2921,15984,13063
4,Sorry about that.,f1a39f08-6e76-4662-b714-d0fa1e561a06,17901,17928,27
5,"Agenda in-house. We'll get minutes. Thanks, James. We'll get decisions. We'll get actions. Now, this stuff I'm guessing is pretty familiar with because this is really the core of what your product does at the moment. Now, when I look at the minutes, we can probably arguably say that actually in the minutes are the decisions, actions. There may be other things that get captured. So so you're familiar with raid what concept of raid is the kind of framework it's yeah it's an acronym basically so it's risks actions issues dependencies um so the other things that may get captured in here would be risks actions Actions we've got above. Issues. And and this sometimes it's a double D, so they have decisions which already got and dependencies. Now the reason I mention all these things is that if your tool could break this stuff out into a series of actions with due dates, which I think you already do, also Also identify where you've got some risks, issues, dependencies, and the decisions that we've got above. In fact, let's put the decisions down there to make it a bit clearer. And if you could then take those into whatever management tool. So if it's a, if If it's waterfall, we tend to use what we call red logs, which are basically literally that, just logs. If it's Agile, we'll tend to use Agile, sorry, things like Jira, which you're very familiar with, I'm sure, or ADO. you Have you come across ADO? No. Azure DevOps? you come across Azure DevOps? No. Okay, so ADO is Azure DevOps, so it's the Microsoft DevOps platform. So it's the thing with Git repos, pipelines, all of that sort of stuff. They've got a thing called Jira boards, which is basically just another version of ADO boards, which is basically like a Jira board. But if we can take those, put those into a tool, let's say Jira, it can then raise all these risk issues, actions, etc. And then if the AI could also sit across this, it could do chases. Now Jira will already send out automated, and Azure DevOps Portals will also do the same. It'll send out automated choices anyway and say, oh, you've not updated your ticket in a few days, but that's not very 

In [236]:
r.groupby('speaker')['span'].sum().sort_values()

speaker
5062be38-0ba9-4e87-94e3-e3365454f932       10
cf8ef7d6-bca6-4ca6-8c7b-630b4ee98acc    11128
Name: span, dtype: int64

In [237]:
path = f'/app/testdata/{connection_id}.webm'
from audio.audio import *
from pathlib import Path
audio_slicer =await AudioSlicer.from_file(path,'webm')

In [238]:
audio_slicer.slice(22,60)

In [105]:
n = 117
slice_ = [r.loc[n]['min']/100,r.loc[n]['max']/100]
audio_slicer.slice(slice_[0],slice_[1])

In [81]:
r['speaker'].unique()

array(['c91b86b5-e41c-48b3-9a81-2addd33cbc08',
       '361db38b-6a25-48db-b5f3-c04812002c14',
       '73279877-dcb8-4194-8716-d149db6a94e4',
       'c9822d44-6764-40d0-82a2-57cc3530fc9f',
       'c0549776-4c0b-4e55-bf95-d196ea4a56ec',
       'ad6acaad-ac6a-4583-817c-d29d2559c608',
       'c1fa9976-c12d-4ad4-80f7-8e14b0e58c22',
       '2443495f-a52a-46a9-be44-0a1d8678ddd3',
       '258a6be3-80af-4fa7-be2f-0037e526a25f'], dtype=object)